<a href="https://colab.research.google.com/github/yellowflickerbeat/FakeNews_Detection/blob/main/vader(no_lemma).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
#stopwords removes all the repetative words which don't substance to the text
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
#WordNetLemmatizer converts words into their roots considering its context
from sklearn.feature_extraction.text import TfidfVectorizer
#TfidfVectorizer measures importance of diff words
from sklearn.model_selection import train_test_split
#splits data into test and train
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
import nltk
nltk.download('stopwords')
print(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
#mounting the mandatory datasets
news = pd.read_table('/content/train.tsv')
news.shape

(7771, 6)

In [5]:
news = news.rename(columns={'Unnamed:0' : 'id'})
#find out missing values in dataset
news.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
subject,0
date,0
label,0


In [3]:
Y = news['label']
news = news.drop('label', axis=1)
print(Y)

0       1.0
1       0.0
2       1.0
3       0.0
4       0.0
       ... 
7766    0.0
7767    0.0
7768    1.0
7769    0.0
7770    NaN
Name: label, Length: 7771, dtype: float64


In [12]:
Y_test = Y_test.dropna().reset_index(drop=True)
X_test, Y_test = X_test.align(Y_test, join='inner', axis=0)
print(Y_train.isnull().sum(), Y_test.isnull().sum())

1 0


In [13]:
y_train_cleaned = Y_train.dropna()

# Align X_train by keeping only the rows that correspond to the non-NaN values in y_train_cleaned
X_train_cleaned = X_train.loc[y_train_cleaned.index]

# Reset the indices of both X_train_cleaned and y_train_cleaned
X_train_cleaned = X_train_cleaned.reset_index(drop=True)
y_train_cleaned = y_train_cleaned.reset_index(drop=True)

# Now check for null values in the cleaned data
print("y_train_cleaned NaN values:", y_train_cleaned.isnull().sum())
print("X_train_cleaned NaN values:", X_train_cleaned.isnull().sum())
print("Y_test NaN values:", Y_test.isnull().sum())

y_train_cleaned NaN values: 0
X_train_cleaned NaN values: neg         0
neu         0
pos         0
compound    0
dtype: int64
Y_test NaN values: 0


In [5]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.4 MB/s eta 0:00:00


In [10]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
def get_sentiment_features(text):
    scores = analyzer.polarity_scores(text)
    return scores['neg'], scores['neu'], scores['pos'], scores['compound']

# Apply the sentiment analysis function to the dataset
news[['neg', 'neu', 'pos', 'compound']] = news['title'].apply(get_sentiment_features).apply(pd.Series)

# Prepare the feature set and target variable
X = news[['neg', 'neu', 'pos', 'compound']]  # Use VADER sentiment features
#y = news['label']  # 1 for real, 0 for fake

# Train-test split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [16]:
model = LogisticRegression()
model.fit(X_train_cleaned, y_train_cleaned)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(Y_test, y_pred)
print(f"Model Accuracy: {accuracy:.3f}")

Model Accuracy: 0.530
